In [1]:
import numpy as np
import tensorflow as tf
import seaborn as sns
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import shutil
import random

from glob import glob
import cv2                
import matplotlib.pyplot as plt                        
from tqdm import tqdm

from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

from torchvision import datasets
import torchvision.transforms as transforms

import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [2]:
classes_list = os.listdir('../Datasets/cats/images')
print (classes_list)

['Abyssinian', 'American Bobtail', 'American Curl', 'American Shorthair', 'American Wirehair', 'Applehead Siamese', 'Balinese', 'Bengal', 'Birman', 'Bombay', 'British Shorthair', 'Burmese', 'Burmilla', 'Calico', 'Canadian Hairless', 'Chartreux', 'Chausie', 'Chinchilla', 'Cornish Rex', 'Cymric', 'Devon Rex', 'Dilute Calico', 'Dilute Tortoiseshell', 'Domestic Long Hair', 'Domestic Medium Hair', 'Domestic Short Hair', 'Egyptian Mau', 'Exotic Shorthair', 'Extra-Toes Cat - Hemingway Polydactyl', 'Havana', 'Himalayan', 'Japanese Bobtail', 'Javanese', 'Korat', 'LaPerm', 'Maine Coon', 'Manx', 'Munchkin', 'Nebelung', 'Norwegian Forest Cat', 'Ocicat', 'Oriental Long Hair', 'Oriental Short Hair', 'Oriental Tabby', 'Persian', 'Pixiebob', 'Ragamuffin', 'Ragdoll', 'Russian Blue', 'Scottish Fold', 'Selkirk Rex', 'Siamese', 'Siberian', 'Silver', 'Singapura', 'Snowshoe', 'Somali', 'Sphynx - Hairless Cat', 'Tabby', 'Tiger', 'Tonkinese', 'Torbie', 'Tortoiseshell', 'Turkish Angora', 'Turkish Van', 'Tuxedo

In [3]:
root_dir = '../Datasets/cats/images/' # data root path (data origin)
working_dir = 'working_cats/images/'
classes_dir = classes_list #total labels

num_classes = len(classes_list)

val_ratio = 0.15
test_ratio = 0.05

In [4]:
if (os.path.isdir('working_cats/') == False):
    #shutil.rmtree('../working/')
    os.listdir('working_cats/')
    !os.mkdir(working_dir)

In [5]:
if (os.path.isdir('working_cats') == False):
    for cls in classes_dir:
        os.makedirs(working_dir +'train/' + cls)
        os.makedirs(working_dir +'val/' + cls)
        os.makedirs(working_dir +'test/' + cls)


        # Creating partitions of the data after shuffling
        src = root_dir + cls # Folder to copy images from

        allFileNames = os.listdir(src)
        #np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                  [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                                   int(len(allFileNames)* (1 - test_ratio))])


        train_FileNames = [src+ '/' + name for name in train_FileNames.tolist()]
        val_FileNames = [src + '/' +name for name in val_FileNames.tolist()]
        test_FileNames = [src + '/' +name for name in test_FileNames.tolist()]


        # Copy-pasting images
        print(cls)
        for name in train_FileNames:
            shutil.copy(name, working_dir +'train/' + cls)

        for name in val_FileNames:
            shutil.copy(name, working_dir +'val/' + cls)

        for name in test_FileNames:
            shutil.copy(name, working_dir +'test/' + cls)

        print(cls, ' complete.')

In [6]:
import torchvision.models as models
import torch.nn as nn
import torch

## TODO: Specify model architecture 
# Load the pretrained model from pytorch


model_resnet50 = models.resnet50(pretrained=True)

# Freeze training for all "features" layers
#for param in model_transfer.features.parameters():
for param in model_resnet50.parameters():
    param.requires_grad = False    

model_resnet50.fc = nn.Linear(2048, num_classes, bias=True) 
    
for param in model_resnet50.fc.parameters():
    param.requires_grad = True

# print out the model structure
print(model_resnet50)
    
    


model_vgg16 = models.vgg16(pretrained=True)

for param in model_vgg16.parameters():
    param.requires_grad = False    
    
model_vgg16.classifier[6] = nn.Linear(model_vgg16.classifier[3].out_features, num_classes)
model_vgg16.classifier[6].requires_grad = True

# print out the model structure

print(model_vgg16)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\devon/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\devon/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
torch.cuda.can_device_access_peer(0, 1)

if torch.cuda.is_available():
    model_vgg16 = model_vgg16.cuda()
    model_resnet50 = model_resnet50.cuda()
    use_cuda = True
else:
    use_cuda = False
print(use_cuda)

AssertionError: Torch not compiled with CUDA enabled